In [30]:
__author__ = 'dk'
from flowcontainer.extractor import extract
result = extract(r"D:\PyProject\vocation\sequence\traffic_data\微信\PCAPdroid_10_11月_09_34_29.pcap",filter='',extension=['tcp.payload','udp.payload'])




Reading D:\PyProject\vocation\sequence\traffic_data\微信\PCAPdroid_10_11月_09_34_29.pcap...


In [42]:
def pad_or_truncate(some_list, target_len):
    return some_list[:target_len] + [0]*(target_len - len(some_list))


In [45]:
def hex_to_dec(hex_str,target_length):
    dec_list = []
    for i in range(0, len(hex_str), 2):
        dec_list.append(int(hex_str[i:i+2], 16))
    dec_list=pad_or_truncate(dec_list,target_length)
    return dec_list
hex_str = '13970100000100000000000003646e730677656978696e02717103636f6d02636e0000010001'
dec_list = hex_to_dec(hex_str,100)
print(dec_list) # [18, 52, 86, 120, 144, 178, 212, 246]

[19, 151, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 3, 100, 110, 115, 6, 119, 101, 105, 120, 105, 110, 2, 113, 113, 3, 99, 111, 109, 2, 99, 110, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [38]:
import numpy as np

# 尝试提取TCP、UDP流的有效载荷部分

In [37]:
flow_id=0
for key in result:
    value = result[key]
    # packet_id=0
    if 'udp.payload' in value.extension:
        print("这是第{}个flow".format(flow_id))
        # print(value.extension['udp.payload'])
        for item in value.extension['udp.payload']:
            print(item[0])
            print(type(item[0]))
    flow_id+=1

这是第17个flow
13970100000100000000000003646e730677656978696e02717103636f6d02636e0000010001
<class 'str'>
13978180000100050000000003646e730677656978696e02717103636f6d02636e0000010001c00c000100010000008b00042abb837ec00c000100010000008b00042abbb8e8c00c000100010000008b0004dcc25d53c00c000100010000008b0004dcc25b9fc00c000100010000008b00046df4a9ec
<class 'str'>
这是第41个flow
e9d301000001000000000000056d6d62697a047170696302636e0000010001
<class 'str'>
e9d381800001000b00000000056d6d62697a047170696302636e0000010001c00c00050001000000130016056d6d62697a047170696302746302717103636f6d00c02b000500010000010d0016056d6d62697a036d69640674646e73763603636f6d00c04d000500010000002a002c056d6d62697a047170696302636e0573636865640b6c65676f706963312d646b0674646e73763603636f6d00c06f000100010000001c00047d4a05c5c06f000100010000001c000476b41a28c06f000100010000001c000475223277c06f000100010000001c000431777b63c06f000100010000001c000476b43c78c06f000100010000001c000431777b3ac06f000100010000001c000476b41a26c06f000100010000001c00047

# 尝试提取前n个包的m个字节负载，并且短则填充长则截断
- 每一个pcap文件转为一个npy文件，转为二维list
- 然后转为npy，保存，合并代码之前已经写好

In [79]:
def get_payload(pcap,n,m):
    result = extract(pcap,extension=['tcp.payload','udp.payload'])
    # 假设有k个流
    pay_load=[]

    for key in result:
        value=result[key]
        packet_num=0
        if 'tcp.payload' in value.extension:
            # 提取tcp负载
            tcp_payload=[]
            for packet in value.extension['tcp.payload']:
                if packet_num<n:
                    # packet[0]是负载，1是标注该报文在流的顺序
                    load = packet[0]
                    tcp_payload.extend(hex_to_dec(load,m))
                    packet_num+=1
                else:
                    break
            # 当前包数太少，加0
            if packet_num<n:
                tcp_payload = pad_or_truncate(tcp_payload,m*n)
            pay_load.append(tcp_payload)
        elif 'udp.payload' in value.extension:
            # 提取ucp负载
            udp_payload=[]
            for packet in value.extension['udp.payload']:
                if packet_num<n:
                    # packet[0]是负载，1是标注该报文在流的顺序
                    load = packet[0]
                    udp_payload.extend(hex_to_dec(load,m))
                    packet_num+=1
                else:
                    break
            # 当前包数太少，加0
            if packet_num<n:
                udp_payload = pad_or_truncate(udp_payload,m*n)
            pay_load.append(udp_payload)
    pay_load=np.array(pay_load)
    return np.uint8(pay_load)

pcap=r"D:\PyProject\vocation\sequence\traffic_data\微信\PCAPdroid_10_11月_09_34_29.pcap"
# 提取每个流前3个数据包前64字节内容
pay_load= get_payload(pcap,3,64)
print(pay_load)

Reading D:\PyProject\vocation\sequence\traffic_data\微信\PCAPdroid_10_11月_09_34_29.pcap...
[[ 22 241   4 ... 241   4   2]
 [ 80  79  83 ...  47 111  99]
 [ 80  79  83 ... 103 170 168]
 ...
 [ 80  79  83 ...  47 111  99]
 [233 211   1 ...   0   0   0]
 [ 22   3   1 ... 158  53  42]]
[[ 22 241   4 ... 241   4   2]
 [ 80  79  83 ...  47 111  99]
 [ 80  79  83 ... 103 170 168]
 ...
 [ 80  79  83 ...  47 111  99]
 [233 211   1 ...   0   0   0]
 [ 22   3   1 ... 158  53  42]]


Reading D:\PyProject\vocation\sequence\traffic_data\微信\PCAPdroid_10_11月_09_34_29.pcap...


C:\Users\ll\.conda\envs\torch\lib\site-packages\ipykernel_launcher.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


ValueError: setting an array element with a sequence.

In [28]:
for key in result:
    value = result[key]
    if 'udp.payload' in value.extension:
        pay_load=''
        for item in value.extension['udp.payload']:
            pay_load+=item[0]
        pay_load=pad_or_truncate(pay_load,784)
        print(pay_load)



b3460100000100000000000004626b767605766964656f02717103636f6d0000010001b3468180000100030000000004626b767605766964656f02717103636f6d0000010001c00c000500010000014900240c696e732d6d36736667317276036961730d74656e63656e742d636c6f7564036e657400c02f000100010000001b0004655931dfc02f000100010000001b0004655932d30000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
29280100000100000000000002767605766964656f02717103636f6d000001000129288180000100050000000002767605766964656f02717103636f6d0000010001c00c000500010000002100240c696e732d6769757a68677761036961730d74656e63656e742d636c6f7